In [1]:
import psycopg2
import time
from itertools import chain
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests 
from datetime import datetime, timedelta
import re
import pytz
import dash 
from sklearn.linear_model import LinearRegression
import math as math
import plotly.graph_objs as go
import dash_table
import sys
from urllib.request import urlretrieve
import zipfile
from dateutil.parser import parse
import json
from random import shuffle
import random
import datetime
import os
from sagemaker import get_execution_role
import sagemaker

import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

# import scrython
# import asyncio
# import aiohttp

/opt/conda/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
conn = psycopg2.connect()

cur=conn.cursor()

cur.execute('''SELECT ebayname.cardname FROM 
((SELECT DISTINCT(cardname) FROM transactions) AS ebayname 
INNER JOIN 
(SELECT pio.cardname FROM ((SELECT DISTINCT(cardname) FROM tourninfo) AS pio 
INNER JOIN 
(SELECT DISTINCT(cardname) FROM moderntourninfo) AS mod ON pio.cardname=mod.cardname)) 
as overall ON ebayname.cardname=overall.cardname) ORDER BY ebayname.cardname ASC''')   
    
namerows=cur.fetchall()


In [3]:
sagemaker_session=sagemaker.Session(boto3.session.Session())

In [4]:
cur.execute('''SELECT cardname,carddate,price FROM public.totalgoatbot WHERE cardtype=0 AND cardname IN(SELECT ebayname.cardname FROM 
((SELECT DISTINCT(cardname) FROM transactions) AS ebayname 
INNER JOIN 
(SELECT pio.cardname FROM ((SELECT DISTINCT(cardname) FROM tourninfo) AS pio 
INNER JOIN 
(SELECT DISTINCT(cardname) FROM moderntourninfo) AS mod ON pio.cardname=mod.cardname)) 
as overall ON ebayname.cardname=overall.cardname))''')
goatrows=cur.fetchall()

goatrows=pd.DataFrame(goatrows,columns=["cardname","carddate","price"])
goatrows['carddate'] = goatrows['carddate'].astype('datetime64[ns]') 

overalllist=[]
for cardname in goatrows["cardname"].unique():
    coeflist=[]
    wowee=goatrows['cardname'] == cardname
    regvals=goatrows.loc[np.logical_and(wowee.values,goatrows['carddate']>'2019-10-01')]
    regvals["carddate"]=regvals["carddate"]-regvals.iloc[0]["carddate"]
    regvals["carddate"]=regvals["carddate"].astype('int64')/86400000000000
    regprice=LinearRegression().fit(np.array(regvals.iloc[math.floor(len(regvals))-4:len(regvals)]["carddate"]).reshape(-1, 1),regvals.iloc[math.floor(len(regvals))-4:len(regvals)]["price"])
    pricecoef=regprice.coef_[0]
    coeflist.append(pricecoef)
    coeflist.append(cardname)
    overalllist.append(coeflist[::-1])
    
# coefdf=pd.DataFrame(overalllist,columns=["cardname","mecoef1","avgcoef1","medcoef.5","avgcoef.5"])
coefdf=pd.DataFrame(overalllist).sort_values(by=1,axis=0)
coefdf.columns=["cardname","slope"]


# Define function using cursor.executemany() to insert the dataframe
def execute_many(conn, datafrm, table):
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES(%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    cur.execute('''DELETE FROM public.slopes''')
    try:
        cursor.executemany(sql, tpls)
        conn.commit()
        print("Data inserted using execute_many() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()
        
execute_many(conn,coefdf,'slopes')


cur.close()
conn.close()

In [5]:
from sagemaker.serializers import IdentitySerializer
class DeepARPredictor(sagemaker.predictor.Predictor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, 
                         #serializer=JSONSerializer(),
                         serializer=IdentitySerializer(content_type="application/json"),
                         **kwargs)
        
    def predict(self, ts, cat=None, dynamic_feat=None, 
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + ts.index.freq
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)
    
    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)
        
        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')
    
    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
        prediction_index = pd.date_range(start=prediction_time, freq=freq, periods=prediction_length)
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq
        
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]        

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat        
    return obj

In [6]:
training_job_name=

attached_estimator = sagemaker.estimator.Estimator.attach(training_job_name, sagemaker_session=sagemaker_session)

predictor=attached_estimator.deploy(    initial_instance_count=1,
    instance_type='ml.m5.large',
    predictor_cls=DeepARPredictor)   
##pass endpoint name into above


2021-02-15 20:04:47 Starting - Preparing the instances for training
2021-02-15 20:04:47 Downloading - Downloading input data
2021-02-15 20:04:47 Training - Training image download completed. Training in progress.
2021-02-15 20:04:47 Uploading - Uploading generated training model
2021-02-15 20:04:47 Completed - Training job completed
-------------!

In [56]:
from jupyter_dash import JupyterDash
import json
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

PAGE_SIZE=15

app = JupyterDash(__name__)


app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])


page_1_layout = html.Div([dcc.Dropdown(
        id="worl",
        options=[{"label": x, "value": x} 
                 for x in ['Winners','Losers']],
        value='Winners',
        clearable=False,
    ),
    html.Div(dash_table.DataTable(
        id='datatable-paging-page-count',
        columns=[
            {"name": i, "id": i} for i in sorted(["cardname","slope"])
        ],
        page_current=0,
        page_size=PAGE_SIZE,
        page_action='custom'
    )),
    html.Div(id='page-1-content'),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2'),
])

@app.callback(
    Output('datatable-paging-page-count', 'data'),
    Input('datatable-paging-page-count', "page_current"),
    Input('worl',"value"))

def update_table(page_current,value,page_size=20):
    conn = psycopg2.connect()

    cur=conn.cursor()
    if value=='Winners':
        cur.execute('''SELECT MAX(id) FROM public.slopes''')
        max_id=cur.fetchall()
        max_id=max_id[0][0]
        cur.execute('''SELECT cardname,slope FROM public.slopes WHERE id<=%s AND id>%s''',
                    (max_id-page_current*page_size,max_id-(page_current+ 1)*page_size))
        coeftups=cur.fetchall()
        coefdf=pd.DataFrame(coeftups,columns=["cardname","slope"]).iloc[::-1].to_dict('records')
    else:
        cur.execute('''SELECT cardname,slope FROM public.slopes WHERE id>%s AND id<=%s''',
                    (page_current*page_size,(page_current+ 1)*page_size))
        coeftups=cur.fetchall()
        coefdf=pd.DataFrame(coeftups,columns=["cardname","slope"]).to_dict('records')
    
    return coefdf




page_2_layout = html.Div([
    html.Div(dcc.Dropdown(
        id="graphstyle",
        options=[{"label": x, "value": x} 
                 for x in ['MINAVGMED','price0','modratio','pioratio','standratio']],
        value='MINAVGMED',
        clearable=False,
    ))
    ,html.Div(dcc.Dropdown(
        id="ticker",
        options=[{"label": x[0], "value": x[0]} 
                 for x in namerows],
        value='Thoughtseize',
        clearable=False,
    )),
    dcc.Graph(id="time-series-chart"),
    html.Img(id="scryfallimg"),
    html.Div(id='page-2-content'),
    html.Br(),
    dcc.Link('Go to Page 1', href='/page-1'),
])

@app.callback(Output("scryfallimg", 'src'),
              Input("ticker", "value"))
def scryfall_img(ticker):
    url=requests.get('https://api.scryfall.com/cards/search?q='+str(ticker))
    if "card_faces" in url.json()['data'][0]:
        return url.json()['data'][0]['card_faces'][0]['image_uris']['normal']
    else:
        return url.json()['data'][0]['image_uris']['normal']
    


@app.callback(
    Output("time-series-chart", "figure"), 
    Input("ticker", "value"),Input("graphstyle", "value"))
def display_time_series(ticker,graphstyle):


    conn = psycopg2.connect()

    cur=conn.cursor()

     ##updated by user
    cat=0
    freq='D'
    prediction_length=7
    context_length=7
    
    analyname=(ticker,)
    
    utc_now = pytz.utc.localize(datetime.datetime.utcnow())
    pst_now = utc_now.astimezone(pytz.timezone("America/Los_Angeles"))

    start_date_othervarbs=pd.Timestamp("2019-10-16", freq=freq)
    start_date = pd.Timestamp("2019-10-23", freq=freq)
    end_dataset = pd.Timestamp(pst_now.date(), freq=freq)
    end_training = end_dataset-timedelta(days=7)
    mid_dataset=pd.Timestamp("2020-09-15",freq=freq)
    end_testing = end_dataset
    
    cur.execute('''SELECT MIN(carddate),cardtype FROM public.totalgoatbot WHERE cardname=%s GROUP BY cardtype''',analyname)

    min_stuff=cur.fetchall()

    min_df=pd.DataFrame(min_stuff,columns=['carddate','cardtype'])

    min_carddate=min_df.min()['carddate']

    min_cardtype=min_df[min_df['carddate']==min_carddate].min()['cardtype']

    minanalyname=(ticker,min_cardtype)

    cur.execute('''SELECT carddate,price,cardtype FROM public.totalgoatbot WHERE cardname ILIKE %s AND cardtype=%s''',minanalyname)

    goatbotrows=cur.fetchall()

    goatdf=pd.DataFrame(goatbotrows,columns=["carddate","price","cardtype"]).set_index("carddate")

    goatdf.index=pd.to_datetime(goatdf.index,infer_datetime_format=True)

    goatdfarr=[]
    
    goatdfarr.append(goatdf)
    goatdfarr[0]=goatdfarr[0].drop(columns='cardtype')
    goatdfarr[0].columns=['price'+str(0)]


    cur.execute('''SELECT carddate,AVG(pshipq) as AVGE,percentile_cont(0.5) WITHIN GROUP (ORDER BY pshipq) as MED
        FROM (SELECT DISTINCT title,cardname,carddate,(price+shipping)/COALESCE(availablequant,cardquantity,maybecardquantity,1) as pshipq FROM public.transactions
        WHERE possiblybad IS FALSE
        AND ismisprint IS NULL
        AND cardname=%s
        AND isfoil IS FALSE
        AND (cardlanguage='english' or cardlanguage IS NULL)
        AND (cardset IS NULL OR cardset NOT IN('LEA','LEB','U'))
        AND (isemblem IS NULL OR isemblem IS FALSE)
        AND (lotis IS FALSE OR lotis IS NULL)
        AND (isboosterbox IS NULL OR isboosterbox IS FALSE)
        AND cardspecial IS NULL
        AND issleeve IS NULL
        AND isdeck IS NULL
        AND isplaymat IS NULL
        AND isultra IS NULL
        AND (istoken IS NULL OR istoken IS FALSE)
        AND (iscommanderdeck IS NULL OR iscommanderdeck IS FALSE)
        AND isgraded IS NULL
        AND (othercards IS NULL OR transplit IS TRUE)
        AND saletype IN('normal')
        AND price+shipping<100) as foo WHERE pshipq<50 
        GROUP BY carddate ORDER BY carddate ASC''',analyname)

    ebayrows=cur.fetchall()

    #ebaydf=pd.DataFrame(ebayrows,columns=["carddate","avg","med"]) 
    ebaydf=pd.DataFrame(ebayrows,columns=["carddate","AVG","MED"]).set_index("carddate")
    ebaydf['MINAVGMED']=ebaydf.apply(lambda row: np.minimum(row.AVG,row.MED), axis = 1)
    ebaydf=ebaydf.drop(columns=['AVG','MED'])
    ebaydf.index=pd.to_datetime(ebaydf.index,infer_datetime_format=True)

    cur.execute('''SELECT entrydate1 as entrydate,SUM(cardquant)/AVG(aggcount) as ratio FROM(
    SELECT DISTINCT cardname0,entrydate1,cardquant,aggcount,tourntitle FROM
    (SELECT cardname as cardname0,entrydate as entrydate0,cardquant,tourntitle FROM public.tourninfo WHERE cardname ILIKE %s) AS foo1,
    (SELECT entrydate as entrydate1,SUM(cardquant) as aggcount FROM public.tourninfo GROUP BY entrydate) AS bar
    WHERE entrydate0=entrydate1) AS foo
    GROUP BY cardname0,entrydate1''',analyname)

    piotournrows=cur.fetchall()

    piotourndf=pd.DataFrame(piotournrows,columns=["carddate","pioratio"]).set_index("carddate")
    piotourndf=piotourndf.astype({'pioratio': 'float64'})

    piotourndf.index=pd.to_datetime(piotourndf.index,infer_datetime_format=True)

    cur.execute('''SELECT entrydate1 as entrydate,SUM(cardquant)/AVG(aggcount) as ratio FROM(
    SELECT DISTINCT cardname0,entrydate1,cardquant,aggcount,tourntitle FROM
    (SELECT cardname as cardname0,entrydate as entrydate0,cardquant,tourntitle FROM public.moderntourninfo WHERE cardname ILIKE %s) AS foo1,
    (SELECT entrydate as entrydate1,SUM(cardquant) as aggcount FROM public.moderntourninfo GROUP BY entrydate) AS bar
    WHERE entrydate0=entrydate1) AS foo
    GROUP BY cardname0,entrydate1''',analyname)

    moderntournrows=cur.fetchall()

    moderntourndf=pd.DataFrame(moderntournrows,columns=["carddate","modratio"]).set_index("carddate")
    moderntourndf=moderntourndf.astype({'modratio': 'float64'})

    moderntourndf.index=pd.to_datetime(moderntourndf.index,infer_datetime_format=True)

    missingdatefix=pd.date_range(start='2019-10-15',end=end_dataset)

    moderntourndf=moderntourndf.reindex(missingdatefix).fillna(0)

    moderntourndf.index.name='carddate'


    cur.execute('''SELECT entrydate1 as entrydate,SUM(cardquant)/AVG(aggcount) as ratio FROM(
    SELECT DISTINCT cardname0,entrydate1,cardquant,aggcount,tourntitle FROM
    (SELECT cardname as cardname0,entrydate as entrydate0,cardquant,tourntitle FROM public.standardtourninfo WHERE cardname ILIKE %s) AS foo1,
    (SELECT entrydate as entrydate1,SUM(cardquant) as aggcount FROM public.standardtourninfo GROUP BY entrydate) AS bar
    WHERE entrydate0=entrydate1) AS foo
    GROUP BY cardname0,entrydate1''',analyname)

    standardtournrows=cur.fetchall()

    if len(standardtournrows)==0:
        standardtourndf=pd.DataFrame([['2020-04-03',0],['2020-04-04',0]],
                                     columns=["carddate","standratio"]).set_index("carddate")
    else:
        standardtourndf=pd.DataFrame(standardtournrows,columns=["carddate","standratio"]).set_index("carddate")

    standardtourndf=standardtourndf.astype({'standratio': 'float64'})
    standardtourndf.index=pd.to_datetime(standardtourndf.index,infer_datetime_format=True)
    print(ebaydf.index.min())
    goatdfarr.insert(0,ebaydf)
    goatdfarr.append(piotourndf)
    goatdfarr.append(moderntourndf)
    goatdfarr.append(standardtourndf)
    overalldf=pd.concat(goatdfarr,axis=1,sort=True)


    overalldf=pd.concat([overalldf['MINAVGMED'].fillna("NaN"),
                         overalldf.loc[:, overalldf.columns.difference(['MINAVGMED'])]
                         .fillna(method='ffill').fillna(method='bfill')],
                        axis=1,join='inner')



    dynamic_feat_arr_series=[]
    
    for col in overalldf.columns[1:]:
        dynamic_feat_arr_series.append(list(overalldf[start_date-timedelta(days=prediction_length):end_dataset][col]))
    if graphstyle=='MINAVGMED':        
        args = {
            "ts": overalldf[start_date:end_dataset]['MINAVGMED'].astype('float64').asfreq(freq),
            "return_samples": False,
            "dynamic_feat":dynamic_feat_arr_series,
            "quantiles": [0.05, 0.5, 0.95],
            "num_samples": 100,
            "cat":namerows.index(analyname)
        }

        sage_preds=predictor.predict(**args)
        print(sage_preds)
        
        overalldf=overalldf[overalldf.index>=ebaydf.index.min()]

        fig = px.line(overalldf, y=graphstyle)
        
        fig.add_trace(go.Scatter(x=sage_preds.index,y=sage_preds['0.5'].values,
                                connectgaps=True, name='<b>Predictions</b>'))
        fig.add_trace(go.Scatter(x=sage_preds.index,y=sage_preds['0.05'].values,
                                connectgaps=True, name='<b>Predictions Lower</b>'))
        fig.add_trace(go.Scatter(x=sage_preds.index,y=sage_preds['0.95'].values,
                                connectgaps=True, name='<b>Predictions Upper</b>'))
        
    else:
        overalldf=overalldf[overalldf.index>=ebaydf.index.min()]

        fig = px.line(overalldf, y=graphstyle)
        
    return fig


@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])

def display_page(pathname):
    if pathname == '/page-2':
        return page_2_layout
    else:
        return page_1_layout

    

app.run_server(mode="external",host='172.17.0.2',port='8050',debug=True)

Dash app running on http://172.17.0.2:8050/
2019-10-21 00:00:00
                 0.95        0.5      0.05
2021-02-16  15.106190  10.978812  5.586014
2021-02-17  17.324198  10.123502  5.198293
2021-02-18  14.446716   9.906779  6.288001
2021-02-19  16.390244   9.180943  3.648102
2021-02-20  18.214828  11.130569  5.505061
2021-02-21  16.226559  10.436534  6.122964
2021-02-22  15.389554  11.762131  7.478086
